In [1]:
import os
import h5py
import time
import warnings
import datetime
import numpy as np

from featureExtraction.pos.postag import *
from featureExtraction.lexicon.lexicon import *

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score

In [2]:
clean = [x for x in os.listdir("../Dataset/Clean") if (x.lower().endswith(".csv"))]
clean.sort()

DATA_CLEAN = ["../Dataset/Clean/" + x for x in os.listdir("../Dataset/Clean") if (x.lower().endswith(".csv"))]
DATA_CLEAN.sort()
len_clean = len(DATA_CLEAN)

In [3]:
warnings.filterwarnings("ignore")

count = 1
fail = 0

f1_list = []
acc_list = []

In [4]:
for i in DATA_CLEAN:
    try:
        print("Running {}/{} ({})".format(count, len_clean, clean[count-1]))
        start_time = time.time()
        
        # Feature Extraction
        result_fe1, label = run_postag(i)
        result_fe2, label, feat_name = run_lexiconCombined_tweet(i)
        result_fe = np.concatenate((result_fe1, result_fe2), axis=1)
        fe_time = time.time()

        # prepare classifier
        cv = KFold(n_splits=20, random_state=1, shuffle=True)
        logRes = LogisticRegression(random_state=1, max_iter=10000)
        
        # prepare the cross-validation procedure
        acc_score = cross_val_score(logRes, result_fe, label, scoring='f1_micro', cv=cv, n_jobs=-1)
        f1_score = cross_val_score(logRes, result_fe, label, scoring='f1_macro', cv=cv, n_jobs=-1)
        cl_time = time.time()

        print('Accuracy: {} ({:.5f})'.format(np.mean(acc_score), np.std(acc_score)))
        print('F1-Score: {} ({:.5f})'.format(np.mean(f1_score), np.std(f1_score)))
        print('FE Time: {} seconds'.format(fe_time - start_time))
        print('CL time: {} seconds'.format(cl_time - fe_time))
        
        f1_list.append(np.mean(f1_score))
        acc_list.append(np.mean(acc_score))
        count += 1
    except Exception as e:
        count += 1
        fail += 1
        print(i, ": ", e)
        pass
    print("================================================")

print("================================================")
print("Overall Accuracy = {} ({})".format(sum(acc_list)/len(acc_list), np.std(acc_list)))
print("Overall F1-Score = {} ({})".format(sum(f1_list)/len(f1_list), np.std(f1_list)))
print("Failed = {}/{}".format(fail, len_clean))

Running 1/13 (templated_posneg_Dataset_C_HT_4000.csv)
Accuracy: 0.6487499999999999 (0.02801)
F1-Score: 0.6471996859648425 (0.02757)
FE Time: 3.691763162612915 seconds
CL time: 2.012580633163452 seconds
Running 2/13 (templated_posneg_KicauanBerlabel.csv)
Accuracy: 0.79025 (0.08448)
F1-Score: 0.7697254019509882 (0.09886)
FE Time: 0.7518830299377441 seconds
CL time: 1.4319097995758057 seconds
Running 3/13 (templated_posneg_dataset_komentar_instagram_cyberbullying.csv)
Accuracy: 0.7914473684210526 (0.08472)
F1-Score: 0.7858553478093625 (0.08666)
FE Time: 0.7464559078216553 seconds
CL time: 0.06419754028320312 seconds
Running 4/13 (templated_posneg_dataset_tweet_sentimen_tayangan_tv.csv)
Accuracy: 0.8150000000000001 (0.07762)
F1-Score: 0.8076184948622019 (0.08168)
FE Time: 0.3769872188568115 seconds
CL time: 0.08225035667419434 seconds
Running 5/13 (templated_posneg_dataset_tweet_sentiment_cellular_service_provider.csv)
Accuracy: 0.7395238095238096 (0.12529)
F1-Score: 0.7067582990325956 (0.